# Module 02: Causal Inference Fundamentals

**Estimated Time:** 45 minutes

## Learning Objectives

By the end of this module, you will be able to:

1. ✅ Distinguish correlation from causation
2. ✅ Identify and understand confounding variables
3. ✅ Apply Bradford Hill criteria for causal inference
4. ✅ Create and interpret Directed Acyclic Graphs (DAGs)
5. ✅ Understand different causal scenarios (mediation, colliders)
6. ✅ Know when causal claims are justified

## Why Causal Inference Matters

**Correlation Question:** "Are X and Y related?"

**Causal Question:** "Does X cause Y?"

### The Danger of Confusing Correlation with Causation

**Famous Examples:**
- Ice cream sales correlate with drowning deaths
- Shoe size correlates with reading ability in children
- Number of fire fighters correlates with fire damage

**None of these are causal!** All have confounders:
- Ice cream & drowning: Temperature (summer)
- Shoe size & reading: Age
- Firefighters & damage: Fire severity

### Why This Matters in Research

**Policy decisions need causation:**
- Does this drug *cause* improvement?
- Does education *cause* higher income?
- Does exercise *cause* better health?

Correlation alone can mislead!

In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

warnings.filterwarnings("ignore")

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# Random seed
np.random.seed(42)

# Create output directory
import os

output_dir = "outputs/module02"
os.makedirs(output_dir, exist_ok=True)

print("✅ Module 02: Causal Inference Fundamentals - Setup Complete!")
print(f"📁 Outputs will be saved to: {output_dir}")

---

## Part 1: Correlation vs Causation

### Three Possible Explanations for Correlation

When X and Y are correlated, there are several possible explanations:

#### 1. X → Y (X causes Y)
Example: Smoking → Lung cancer

#### 2. Y → X (Y causes X - reverse causation)
Example: Does depression cause unemployment, or does unemployment cause depression?

#### 3. Z → X and Z → Y (Confounding)
Example: Temperature causes both ice cream sales and drowning

#### 4. Coincidence
Spurious correlation by chance

### The Gold Standard: Randomized Controlled Trial (RCT)

**Why RCTs establish causation:**
- Random assignment breaks all confounding
- Only thing that differs is the treatment
- Any difference in outcomes must be due to treatment

**When RCTs aren't possible:**
- Unethical (e.g., randomly assign people to smoke)
- Impractical (e.g., randomly assign education levels)
- Too expensive or time-consuming

**Then we need observational causal inference!**

In [ ]:
# Demonstrate spurious correlation

# Generate confounder (temperature)
n = 100
temperature = np.random.uniform(60, 100, n)  # Fahrenheit

# Both ice cream and drowning depend on temperature
ice_cream_sales = 10 + 0.5 * temperature + np.random.normal(0, 5, n)
drowning_deaths = -5 + 0.3 * temperature + np.random.normal(0, 3, n)

# Create DataFrame
df = pd.DataFrame(
    {
        "Temperature": temperature,
        "Ice_Cream_Sales": ice_cream_sales,
        "Drowning_Deaths": drowning_deaths,
    }
)

# Correlation between ice cream and drowning
corr = df["Ice_Cream_Sales"].corr(df["Drowning_Deaths"])

print("SPURIOUS CORRELATION EXAMPLE")
print("=" * 80)
print(f"\nCorrelation between ice cream sales and drowning: r = {corr:.3f}")
print(f"p-value: {stats.pearsonr(df['Ice_Cream_Sales'], df['Drowning_Deaths'])[1]:.4f}")
print("\n⚠️  Highly significant correlation!")
print("\nBut wait...")
print("\nCorrelation with temperature:")
print(f"  Ice cream & temp: r = {df['Ice_Cream_Sales'].corr(df['Temperature']):.3f}")
print(f"  Drowning & temp: r = {df['Drowning_Deaths'].corr(df['Temperature']):.3f}")
print("\n💡 Temperature is the CONFOUNDER!")
print("   Both ice cream and drowning increase with temperature")
print("   Ice cream doesn't CAUSE drowning!")

In [ ]:
# Visualize the spurious correlation

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Ice cream vs Drowning (misleading!)
axes[0].scatter(df["Ice_Cream_Sales"], df["Drowning_Deaths"], alpha=0.6)
axes[0].set_xlabel("Ice Cream Sales", fontsize=11)
axes[0].set_ylabel("Drowning Deaths", fontsize=11)
axes[0].set_title(f"Misleading Correlation\nr = {corr:.3f}", fontsize=12, fontweight="bold")
z = np.polyfit(df["Ice_Cream_Sales"], df["Drowning_Deaths"], 1)
p = np.poly1d(z)
axes[0].plot(df["Ice_Cream_Sales"], p(df["Ice_Cream_Sales"]), "r--", alpha=0.8)

# Plot 2: Temperature vs Ice cream
scatter = axes[1].scatter(
    df["Temperature"], df["Ice_Cream_Sales"], c=df["Temperature"], cmap="YlOrRd", alpha=0.6
)
axes[1].set_xlabel("Temperature (°F)", fontsize=11)
axes[1].set_ylabel("Ice Cream Sales", fontsize=11)
axes[1].set_title("True Cause: Temperature → Ice Cream", fontsize=12, fontweight="bold")

# Plot 3: Temperature vs Drowning
scatter2 = axes[2].scatter(
    df["Temperature"], df["Drowning_Deaths"], c=df["Temperature"], cmap="YlOrRd", alpha=0.6
)
axes[2].set_xlabel("Temperature (°F)", fontsize=11)
axes[2].set_ylabel("Drowning Deaths", fontsize=11)
axes[2].set_title("True Cause: Temperature → Drowning", fontsize=12, fontweight="bold")

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "spurious_correlation.png"), dpi=300, bbox_inches="tight")
plt.show()

print("\n💡 This demonstrates why correlation ≠ causation!")

---

## Part 2: Confounding Variables

### What is a Confounder?

**A confounder is a variable that:**
1. Affects both the exposure (X) and outcome (Y)
2. Is NOT on the causal pathway between X and Y
3. Creates a spurious association

### Classic Example: Coffee and Lung Cancer

Early studies found: Coffee drinkers had higher lung cancer rates

**Confounder: Smoking**
- Smokers drink more coffee
- Smoking causes lung cancer
- Coffee → lung cancer association is spurious!

```
        Smoking (Confounder)
          /              \
         ↓                ↓
      Coffee         Lung Cancer
```

### Identifying Confounders

Ask three questions:
1. **Associated with exposure?** Does Z relate to X?
2. **Associated with outcome?** Does Z relate to Y?
3. **Not on causal path?** Is Z not caused by X?

If YES to all three → It's a confounder!

### Controlling for Confounders

**Methods:**
1. **Randomization** (RCT) - breaks all confounding
2. **Stratification** - analyze within confounder levels
3. **Matching** - match on confounder values
4. **Regression adjustment** - include confounder in model
5. **Propensity score methods** - advanced technique

In [ ]:
# Demonstrate confounding and adjustment

np.random.seed(42)
n = 500

# Generate confounder: Age
age = np.random.uniform(20, 70, n)

# Treatment assignment influenced by age (older people more likely to get treatment)
# This is confounding! Treatment is not random
prob_treatment = (age - 20) / 50  # Probability increases with age
treatment = np.random.binomial(1, prob_treatment)

# Outcome (e.g., blood pressure) influenced by:
# 1. Age (older = higher BP)
# 2. Treatment (reduces BP by 10 points)
blood_pressure = 90 + 0.5 * age - 10 * treatment + np.random.normal(0, 5, n)

# Create DataFrame
df_confound = pd.DataFrame({"Age": age, "Treatment": treatment, "Blood_Pressure": blood_pressure})

# Naive comparison (ignoring confounder)
naive_diff = (
    df_confound[df_confound["Treatment"] == 1]["Blood_Pressure"].mean()
    - df_confound[df_confound["Treatment"] == 0]["Blood_Pressure"].mean()
)

print("CONFOUNDING DEMONSTRATION")
print("=" * 80)
print("\nScenario: Does treatment reduce blood pressure?")
print("Confounder: Age (older people more likely to get treatment AND have higher BP)")

print("\n1. NAIVE ANALYSIS (ignoring age):")
print(
    f"   Treated group mean BP: {df_confound[df_confound['Treatment']==1]['Blood_Pressure'].mean():.1f}"
)
print(
    f"   Control group mean BP: {df_confound[df_confound['Treatment']==0]['Blood_Pressure'].mean():.1f}"
)
print(f"   Difference: {naive_diff:.1f} points")
print("   ⚠️  Treated group has HIGHER BP! Does treatment make it worse?")
print("   No! This is confounding - treated group is older!")

# Adjusted comparison (controlling for age)
from scipy.stats import linregress

# Linear regression: BP ~ Treatment + Age
X_reg = df_confound[["Treatment", "Age"]].values
y_reg = df_confound["Blood_Pressure"].values

# Add intercept
X_reg = np.column_stack([np.ones(len(X_reg)), X_reg])

# Coefficients
coeffs = np.linalg.lstsq(X_reg, y_reg, rcond=None)[0]
treatment_effect = coeffs[1]

print("\n2. ADJUSTED ANALYSIS (controlling for age):")
print(f"   Treatment effect: {treatment_effect:.1f} points")
print(f"   ✅ After adjusting for age, treatment REDUCES BP by ~10 points!")
print(f"   This matches the true causal effect we simulated.")

---

## Part 3: Bradford Hill Criteria for Causation

### Historical Context

In 1965, Sir Austin Bradford Hill proposed criteria for judging whether an observed association is likely causal.

**Important:** These are GUIDELINES, not rules. No single criterion is necessary or sufficient.

### The Nine Criteria

#### 1. **Strength of Association**
- Stronger associations more likely causal
- Example: Smoking has very strong association with lung cancer (RR > 10)

#### 2. **Consistency**
- Replicated across different studies, populations, settings
- Example: Smoking-cancer link found worldwide

#### 3. **Specificity**
- Specific exposure → specific outcome
- Example: Asbestos → mesothelioma (rare cancer)
- Note: NOT required for causation (smoking causes many diseases)

#### 4. **Temporality**
- **REQUIRED**: Cause must precede effect
- Example: Exposure to carcinogen before cancer develops

#### 5. **Biological Gradient (Dose-Response)**
- More exposure → stronger effect
- Example: More cigarettes per day → higher cancer risk

#### 6. **Plausibility**
- Biologically plausible mechanism
- Example: Carcinogens damage DNA → cancer
- Limited by current knowledge

#### 7. **Coherence**
- Consistent with known facts about disease
- Doesn't contradict existing knowledge

#### 8. **Experiment**
- Experimental evidence (RCT or natural experiment)
- Gold standard when available

#### 9. **Analogy**
- Similar causes have similar effects
- Example: If one drug in a class is teratogenic, others may be too

### Applying Bradford Hill

**Strong causal evidence when:**
- ✅ Temporality established
- ✅ Strong, consistent association
- ✅ Dose-response relationship
- ✅ Plausible mechanism
- ✅ Experimental support

**Weaker evidence when:**
- ❓ Only cross-sectional data (can't establish temporality)
- ❓ Weak or inconsistent associations
- ❓ No dose-response
- ❓ Many plausible confounders

In [ ]:
# Bradford Hill Criteria Checklist Generator


def bradford_hill_checklist():
    """
    Generate a Bradford Hill criteria evaluation checklist.
    """
    criteria = {
        "Criterion": [
            "1. Strength",
            "2. Consistency",
            "3. Specificity",
            "4. Temporality*",
            "5. Dose-Response",
            "6. Plausibility",
            "7. Coherence",
            "8. Experiment",
            "9. Analogy",
        ],
        "Question": [
            "Is the association strong (large effect size)?",
            "Is it replicated across studies/populations?",
            "Is the effect specific to this exposure-outcome pair?",
            "Does the cause precede the effect? (REQUIRED)",
            "Is there a dose-response relationship?",
            "Is there a plausible biological mechanism?",
            "Does it fit with existing knowledge?",
            "Is there experimental evidence (RCT, etc.)?",
            "Do similar exposures cause similar effects?",
        ],
        "Met": [""] * 9,
        "Evidence": [""] * 9,
    }

    return pd.DataFrame(criteria)


checklist = bradford_hill_checklist()

print("BRADFORD HILL CRITERIA CHECKLIST")
print("=" * 80)
print("\nUse this checklist to evaluate causal evidence:")
print("\n" + checklist[["Criterion", "Question"]].to_string(index=False))
print("\n* Temporality is the ONLY essential criterion")
print("\n💡 Save this checklist and use it for your research!")

# Save template
checklist.to_csv(os.path.join(output_dir, "bradford_hill_checklist.csv"), index=False)
print(f"\n✅ Checklist saved to {output_dir}/bradford_hill_checklist.csv")

---

## Part 4: Directed Acyclic Graphs (DAGs)

### What are DAGs?

**Directed Acyclic Graphs** are visual tools for representing causal assumptions.

**Components:**
- **Nodes**: Variables
- **Arrows**: Causal relationships (X → Y means "X causes Y")
- **Acyclic**: No loops (can't go back to where you started)

### Basic DAG Structures

#### 1. Chain (Mediation)
```
X → M → Y
```
- X causes M, M causes Y
- M is a **mediator**
- Example: Exercise → Weight Loss → Blood Pressure

#### 2. Fork (Confounding)
```
    Z
   / \
  ↓   ↓
  X   Y
```
- Z causes both X and Y
- Z is a **confounder**
- Example: Age → Both Treatment & Outcome

#### 3. Collider
```
  X   Y
   \ /
    ↓
    C
```
- X and Y both cause C
- C is a **collider**
- Example: Talent & Beauty → Celebrity

### The Rules of DAGs

#### 1. Confounders: CONTROL for them
```
    Z (confounder)
   / \
  ↓   ↓
  X   Y
```
Must adjust for Z to get causal effect of X on Y

#### 2. Mediators: DON'T control for them
```
X → M → Y
```
If you control for M, you block the causal path!

#### 3. Colliders: DON'T control for them
```
  X   Y
   \ /
    ↓
    C
```
Controlling for C creates spurious association between X and Y!

### Collider Bias (Berkson's Paradox)

**Example:** Are talented actors less attractive?

Look at celebrities (who need talent OR beauty):
```
  Talent    Beauty
      \      /
       ↓    ↓
      Celebrity
```

Among celebrities, talent and beauty appear negatively correlated!
- Very talented → can be less attractive
- Very attractive → can be less talented

**But this is bias!** In general population, no such correlation exists.

In [ ]:
# Demonstrate collider bias

np.random.seed(42)
n = 1000

# Generate talent and beauty (uncorrelated in general population)
talent = np.random.normal(50, 15, n)
beauty = np.random.normal(50, 15, n)

# Become celebrity if talent OR beauty is high
# Celebrity = collider (caused by both talent and beauty)
celebrity_score = talent + beauty
celebrity = celebrity_score > np.percentile(celebrity_score, 90)  # Top 10%

# Check correlation
corr_general = np.corrcoef(talent, beauty)[0, 1]
corr_celebrity = np.corrcoef(talent[celebrity], beauty[celebrity])[0, 1]

print("COLLIDER BIAS DEMONSTRATION")
print("=" * 80)
print("\nScenario: Talent and Beauty as paths to Celebrity")
print("\nDAG:")
print("    Talent    Beauty")
print("        \\      /")
print("         ↓    ↓")
print("       Celebrity (COLLIDER)")

print(f"\nGeneral population (n={n}):")
print(f"  Correlation between talent & beauty: r = {corr_general:.3f}")
print(f"  (Essentially zero - they're independent!)")

print(f"\nAmong celebrities only (n={sum(celebrity)}):")
print(f"  Correlation between talent & beauty: r = {corr_celebrity:.3f}")
print(f"  ⚠️  NEGATIVE correlation!")

print("\n💡 This is COLLIDER BIAS (Berkson's Paradox):")
print("   Conditioning on a collider creates spurious association")
print("   Among celebrities, very talented people can be less attractive")
print("   But this doesn't mean talent and beauty are negatively related!")

In [ ]:
# Visualize collider bias

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# General population
ax1.scatter(talent, beauty, alpha=0.3, s=20)
ax1.set_xlabel("Talent", fontsize=12)
ax1.set_ylabel("Beauty", fontsize=12)
ax1.set_title(
    f"General Population\nr = {corr_general:.3f} (no correlation)", fontsize=13, fontweight="bold"
)
ax1.grid(True, alpha=0.3)

# Celebrities only
ax2.scatter(talent[celebrity], beauty[celebrity], alpha=0.6, s=40, color="red")
ax2.scatter(talent[~celebrity], beauty[~celebrity], alpha=0.1, s=10, color="gray")
ax2.set_xlabel("Talent", fontsize=12)
ax2.set_ylabel("Beauty", fontsize=12)
ax2.set_title(
    f"Conditioning on Celebrity (Collider)\nr = {corr_celebrity:.3f} (negative!)",
    fontsize=13,
    fontweight="bold",
)
ax2.grid(True, alpha=0.3)

# Add regression line for celebrities
z = np.polyfit(talent[celebrity], beauty[celebrity], 1)
p = np.poly1d(z)
ax2.plot(talent[celebrity], p(talent[celebrity]), "b--", linewidth=2, alpha=0.8)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "collider_bias.png"), dpi=300, bbox_inches="tight")
plt.show()

print("\n💡 The negative slope in the right plot is ENTIRELY due to selection bias!")

---

## Part 5: When Can We Claim Causation?

### The Causal Hierarchy

**From weakest to strongest evidence:**

#### Level 1: Cross-Sectional Association
- Single timepoint, observational
- Can establish correlation
- **Cannot establish causation** (no temporality)
- Example: Survey showing coffee drinkers have lower depression

#### Level 2: Longitudinal Association
- Multiple timepoints, observational
- Can establish temporality
- Still subject to confounding
- Example: Following coffee drinkers over time

#### Level 3: Natural Experiment
- "As-if random" assignment by nature/policy
- Quasi-experimental evidence
- Example: Coffee rationing during war

#### Level 4: Randomized Controlled Trial
- Random assignment to conditions
- Gold standard for causation
- Example: Randomly assign coffee vs control

### Strengthening Causal Claims

**Even without RCT, you can strengthen claims by:**

1. **Measure confounders** and adjust for them
2. **Longitudinal data** to establish temporality
3. **Dose-response** relationship
4. **Mechanism** evidence (mediators)
5. **Sensitivity analyses** (test robustness)
6. **Negative controls** (falsification tests)
7. **Instrumental variables** (advanced technique)

### Language Matters!

**Observational study:**
- ✅ "X is associated with Y"
- ✅ "X predicts Y"
- ✅ "X correlates with Y"
- ❌ "X causes Y"
- ❌ "X leads to Y"

**RCT or strong causal evidence:**
- ✅ "X causes Y"
- ✅ "X has a causal effect on Y"
- ✅ "X leads to changes in Y"

**Be honest about limitations!**

In [ ]:
# Create a decision tree for causal claims


def causal_claim_decision_tree():
    """
    Decision tree for determining appropriate causal language.
    """
    print("CAUSAL CLAIM DECISION TREE")
    print("=" * 80)
    print("\nUse this to decide what you can claim:")
    print("\n1. Was there RANDOM ASSIGNMENT to conditions?")
    print("   YES → Go to #2")
    print("   NO → Go to #3")

    print("\n2. RCT conducted properly?")
    print("   YES → ✅ You can make CAUSAL claims")
    print("         Language: 'X causes Y', 'X has a causal effect on Y'")
    print("   NO → Go to #3")

    print("\n3. Is this observational data?")
    print("   Cross-sectional → ⚠️  Can only claim ASSOCIATION")
    print("                      Language: 'X is associated with Y'")
    print("   Longitudinal → Go to #4")

    print("\n4. Did exposure precede outcome (temporality)?")
    print("   NO → ⚠️  Cannot claim causation")
    print("   YES → Go to #5")

    print("\n5. Did you control for confounders?")
    print("   NO → ⚠️  Weak evidence, use cautious language")
    print("        'X is associated with Y, but causation cannot be established'")
    print("   YES → Go to #6")

    print("\n6. Do you have multiple lines of evidence?")
    print("   (dose-response, mechanism, Bradford Hill criteria)")
    print("   YES → ⚠️  Moderate evidence")
    print("         Language: 'Results suggest X may cause Y'")
    print("                   'Evidence is consistent with a causal effect'")
    print("   NO → ⚠️  Weak evidence")

    print("\n" + "=" * 80)
    print("💡 When in doubt, be conservative with causal language!")
    print("   It's better to understate than overstate causal claims.")


causal_claim_decision_tree()

---

## Practice Exercises

### Exercise 1: Identify the Confounder

**Scenario:** A study finds that people who drink wine have better heart health.

**Questions:**
1. What are potential confounders?
2. Draw a DAG showing wine, heart health, and at least two confounders
3. How would you control for these confounders?

### Exercise 2: Apply Bradford Hill Criteria

**Scenario:** You're evaluating whether screen time causes sleep problems in children.

**Your data:**
- Strong correlation (r = 0.6)
- Replicated in 20 studies
- Longitudinal: More screen time predicts future sleep problems
- Dose-response: More hours = worse sleep
- Mechanism: Blue light suppresses melatonin

**Task:** Use Bradford Hill checklist to evaluate causal evidence

### Exercise 3: Spot the Collider

**Scenario:** A hospital study finds that seat belt use is associated with HIGHER injury severity.

**Questions:**
1. Is this evidence that seat belts are dangerous?
2. What's the collider here?
3. Draw a DAG to explain this paradox

**Hint:** Think about who ends up in the hospital!

### Exercise 4: Appropriate Language

**For each study, write appropriate conclusions:**

1. Cross-sectional survey: Meditation correlated with lower stress
2. Longitudinal study: Exercise at T1 predicts weight loss at T2, controlling for baseline weight and diet
3. RCT: Randomly assigned meditation vs control, meditation group had lower stress

**What language should you use for each?**

---

## Summary and Key Takeaways

### 🎯 What We Learned

1. **Correlation ≠ Causation**
   - Multiple explanations for correlation
   - Confounding is common
   - Need additional evidence for causation

2. **Confounding Variables**
   - Affect both exposure and outcome
   - Create spurious associations
   - Must control for them

3. **Bradford Hill Criteria**
   - Framework for evaluating causal evidence
   - Temporality is essential
   - Other criteria strengthen case

4. **Directed Acyclic Graphs (DAGs)**
   - Visual tool for causal assumptions
   - Three structures: chains, forks, colliders
   - Rules for what to control

5. **Causal Inference Standards**
   - RCTs are gold standard
   - Observational studies can suggest causation
   - Language matters!

### 📚 Key Principles

| Principle | Rule |
|-----------|------|
| **Confounders** | CONTROL for them |
| **Mediators** | DON'T control (blocks causal path) |
| **Colliders** | DON'T control (creates bias) |
| **Temporality** | ESSENTIAL for causation |
| **Randomization** | Breaks all confounding |

### 🚀 Next Steps

1. **Practice**: Work through the exercises
2. **Draw DAGs**: For your own research questions
3. **Evaluate Papers**: Apply Bradford Hill to published studies
4. **Be Cautious**: Don't overclaim causation
5. **Prepare** for Module 03: Advanced Experimental Designs

### 💡 Remember

> "Association is what we observe. Causation is what we want to know."

The bridge from correlation to causation requires careful thought and strong evidence!

---

## Additional Resources

### Books
- "The Book of Why" by Judea Pearl
- "Causal Inference: The Mixtape" by Cunningham (free online)
- "Mostly Harmless Econometrics" by Angrist & Pischke

### Software
- **DAGitty**: Draw and analyze DAGs (dagitty.net)
- **DoWhy**: Python causal inference library
- **CausalML**: Machine learning for causal inference

### Online Courses
- Coursera: "A Crash Course in Causality"
- edX: "Causal Inference"

### Papers
- Bradford Hill (1965): "The Environment and Disease: Association or Causation?"
- Pearl (2009): "Causal inference in statistics: An overview"

---

**Next Module:** [03_advanced_experimental_designs.ipynb](03_advanced_experimental_designs.ipynb) - Learn quasi-experiments, within-subjects designs, and real-world research strategies!

---

*Last updated: 2024*